In [4]:
import math
import random

from collections import deque
import gym
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd

from qpth.qp import QPFunction

%matplotlib inline

Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda()
Parameter = lambda *args, **kwargs: nn.parameter.Parameter(*args, **kwargs).cuda()

In [5]:
class Continuous_MountainCarEnv(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 30
    }

    def __init__(self, goal_velocity = 0):
        self.min_action = -1.0
        self.max_action = 1.0
        self.min_position = -1.2
        self.max_position = 0.6
        self.max_speed = 0.07
        self.goal_position = 0.45 # was 0.5 in gym, 0.45 in Arnaud de Broissia's version
        self.goal_velocity = goal_velocity
        self.power = 0.0015

        self.low_state = np.array([self.min_position, -self.max_speed])
        self.high_state = np.array([self.max_position, self.max_speed])

        self.viewer = None

        self.action_space = spaces.Box(low=self.min_action, high=self.max_action,
                                       shape=(1,), dtype=np.float32)
        self.observation_space = spaces.Box(low=self.low_state, high=self.high_state,
                                            dtype=np.float32)

        self.seed()
        self.reset()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):

        position = self.state[0]
        velocity = self.state[1]
        force = min(max(action[0], -1.0), 1.0)

        velocity += force*self.power -0.0025 * math.cos(3*position)
        if (velocity > self.max_speed): velocity = self.max_speed
        if (velocity < -self.max_speed): velocity = -self.max_speed
        position += velocity
        if (position > self.max_position): position = self.max_position
        if (position < self.min_position): position = self.min_position
        if (position==self.min_position and velocity<0): velocity = 0

        done = bool(position >= self.goal_position and velocity >= self.goal_velocity)

        reward = 0
        if done:
            reward = 100.0
        reward-= math.pow(action[0],2)*0.1

        self.state = np.array([position, velocity])
        return self.state, reward, done, {}

    def reset(self):
        self.state = np.array([self.np_random.uniform(low=-0.6, high=-0.4), 0])
        return np.array(self.state)

#    def get_state(self):
#        return self.state

    def _height(self, xs):
        return np.sin(3 * xs)*.45+.55

    def render(self, mode='human'):
        screen_width = 600
        screen_height = 400

        world_width = self.max_position - self.min_position
        scale = screen_width/world_width
        carwidth=40
        carheight=20


        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)
            xs = np.linspace(self.min_position, self.max_position, 100)
            ys = self._height(xs)
            xys = list(zip((xs-self.min_position)*scale, ys*scale))

            self.track = rendering.make_polyline(xys)
            self.track.set_linewidth(4)
            self.viewer.add_geom(self.track)

            clearance = 10

            l,r,t,b = -carwidth/2, carwidth/2, carheight, 0
            car = rendering.FilledPolygon([(l,b), (l,t), (r,t), (r,b)])
            car.add_attr(rendering.Transform(translation=(0, clearance)))
            self.cartrans = rendering.Transform()
            car.add_attr(self.cartrans)
            self.viewer.add_geom(car)
            frontwheel = rendering.make_circle(carheight/2.5)
            frontwheel.set_color(.5, .5, .5)
            frontwheel.add_attr(rendering.Transform(translation=(carwidth/4,clearance)))
            frontwheel.add_attr(self.cartrans)
            self.viewer.add_geom(frontwheel)
            backwheel = rendering.make_circle(carheight/2.5)
            backwheel.add_attr(rendering.Transform(translation=(-carwidth/4,clearance)))
            backwheel.add_attr(self.cartrans)
            backwheel.set_color(.5, .5, .5)
            self.viewer.add_geom(backwheel)
            flagx = (self.goal_position-self.min_position)*scale
            flagy1 = self._height(self.goal_position)*scale
            flagy2 = flagy1 + 50
            flagpole = rendering.Line((flagx, flagy1), (flagx, flagy2))
            self.viewer.add_geom(flagpole)
            flag = rendering.FilledPolygon([(flagx, flagy2), (flagx, flagy2-10), (flagx+25, flagy2-5)])
            flag.set_color(.8,.8,0)
            self.viewer.add_geom(flag)

        pos = self.state[0]
        self.cartrans.set_translation((pos-self.min_position)*scale, self._height(pos)*scale)
        self.cartrans.set_rotation(math.cos(3 * pos))

        return self.viewer.render(return_rgb_array = mode=='rgb_array')

    def close(self):
        if self.viewer:
            self.viewer.close()
            self.viewer = None

In [6]:
from gym import spaces, logger
from gym.utils import seeding
import numpy as np

class MyEnv(gym.Env):
    """
    Able to build required model based on gym.
    Example: CartPoleEnv
    Description:
        A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum starts upright, and the goal is to prevent it from falling over by increasing and reducing the cart's velocity.

    Source:
        This environment corresponds to the version of the cart-pole problem described by Barto, Sutton, and Anderson

    Observation: 
        Type: Box(4)
        Num	Observation                 Min         Max
        0	Cart Position             -4.8            4.8
        1	Cart Velocity             -Inf            Inf
        2	Pole Angle                 -24 deg        24 deg
        3	Pole Velocity At Tip      -Inf            Inf
        
    Actions:
        Type: Discrete(2)
        Num	Action
        0	Push cart to the left
        1	Push cart to the right
        
        Note: The amount the velocity that is reduced or increased is not fixed; it depends on the angle the pole is pointing. This is because the center of gravity of the pole increases the amount of energy needed to move the cart underneath it

    Reward:
        Reward is 1 for every step taken, including the termination step

    Starting State:
        All observations are assigned a uniform random value in [-0.05..0.05]

    Episode Termination:
        Pole Angle is more than 12 degrees
        Cart Position is more than 2.4 (center of the cart reaches the edge of the display)
        Episode length is greater than 200
        Solved Requirements
        Considered solved when the average reward is greater than or equal to 195.0 over 100 consecutive trials.
    """
    
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second' : 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5 # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 10.0
        self.tau = 0.02  # seconds between state updates
        self.kinematics_integrator = 'euler'

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation is still within bounds
        high = np.array([
            self.x_threshold * 2,
            np.finfo(np.float32).max,
            self.theta_threshold_radians * 2,
            np.finfo(np.float32).max])

        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        """ Excutes one constant action to the model for one constant period.
        The return is: np.array(self.state), reward, done, {}
        where: 
            np.array(self.state): the next state
            reward: r(s,a)
            done: if the next state is the terminal state
        """
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        state = self.state
        x, x_dot, theta, theta_dot = state
        force = self.force_mag if action==1 else -self.force_mag
        costheta = -math.cos(theta)
        sintheta = -math.sin(theta)
#         temp = (force + self.polemass_length * theta_dot * theta_dot * sintheta) / self.total_mass
#         thetaacc = (self.gravity * sintheta - costheta* temp) / (self.length * (4.0/3.0 - self.masspole * costheta * costheta / self.total_mass))
#         xacc  = temp - self.polemass_length * thetaacc * costheta / self.total_mass
        # approximate linear model
#         temp = self.masspole * (1/3*self.total_mass + self.masscart) * self.length*self.length
#         thetaacc = self.masspole*self.gravity*self.length*self.total_mass/temp * theta - self.masspole*self.length/temp * self.force_mag
#         xacc = -self.masspole*self.masspole*self.gravity*self.length*self.length/temp * theta + 4.0/3.0*self.masspole*self.length*self.length/temp * self.force_mag
        xacc = -theta+1*self.force_mag
        thetaacc = theta - 1*self.force_mag

        
        if self.kinematics_integrator == 'euler':
            x  = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else: # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x  = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot
        self.state = (x,x_dot,theta,theta_dot)
        done =  x < -self.x_threshold \
                or x > self.x_threshold \
                or theta < -self.theta_threshold_radians \
                or theta > self.theta_threshold_radians
        done = bool(done)

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            # Pole just fell!
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn("You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.")
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state), reward, done, {}

    def reset(self):
        self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state)

    def render(self, mode='human'):
        screen_width = 600
        screen_height = 400

        world_width = self.x_threshold*2
        scale = screen_width/world_width
        carty = 100 # TOP OF CART
        polewidth = 10.0
        polelen = scale * (2 * self.length)
        cartwidth = 50.0
        cartheight = 30.0

        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)
            l,r,t,b = -cartwidth/2, cartwidth/2, cartheight/2, -cartheight/2
            axleoffset =cartheight/4.0
            cart = rendering.FilledPolygon([(l,b), (l,t), (r,t), (r,b)])
            self.carttrans = rendering.Transform()
            cart.add_attr(self.carttrans)
            self.viewer.add_geom(cart)
            l,r,t,b = -polewidth/2,polewidth/2,polelen-polewidth/2,-polewidth/2
            pole = rendering.FilledPolygon([(l,b), (l,t), (r,t), (r,b)])
            pole.set_color(.8,.6,.4)
            self.poletrans = rendering.Transform(translation=(0, axleoffset))
            pole.add_attr(self.poletrans)
            pole.add_attr(self.carttrans)
            self.viewer.add_geom(pole)
            self.axle = rendering.make_circle(polewidth/2)
            self.axle.add_attr(self.poletrans)
            self.axle.add_attr(self.carttrans)
            self.axle.set_color(.5,.5,.8)
            self.viewer.add_geom(self.axle)
            self.track = rendering.Line((0,carty), (screen_width,carty))
            self.track.set_color(0,0,0)
            self.viewer.add_geom(self.track)

            self._pole_geom = pole

        if self.state is None: return None

        # Edit the pole polygon vertex
        pole = self._pole_geom
        l,r,t,b = -polewidth/2,polewidth/2,polelen-polewidth/2,-polewidth/2
        pole.v = [(l,b), (l,t), (r,t), (r,b)]

        x = self.state
        cartx = x[0]*scale+screen_width/2.0 # MIDDLE OF CART
        self.carttrans.set_translation(cartx, carty)
        self.poletrans.set_rotation(-x[2])

        return self.viewer.render(return_rgb_array = mode=='rgb_array')

    def close(self):
        if self.viewer:
            self.viewer.close()
            self.viewer = None


In [7]:
env = MyEnv()
env.reset()
env.close()

In [8]:
class ReplayBuffer(object):
    """ a cyclic buffer of bounded size that holds the 
    transitions observed recently. 
    """
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        state      = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
            
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action, reward, np.concatenate(next_state), done
    
    def __len__(self):
        return len(self.buffer)

In [9]:
# Epsilon-greedy exploration
# The epsilon decreases exponetially as time goes by.
epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 500

epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

In [10]:
class DQN(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.ReLU(),
            nn.Linear(20, num_actions)
        )
        
    def forward(self, x):
        return self.layers(x)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(state).unsqueeze(0)) # adds extra dim at front
            q_value = self.forward(state)
            action  = q_value.max(1)[1]
        else:
            action = random.randrange(env.action_space.n)
        return action

In [11]:
class DQN(nn.Module):
    """Builds the strucre of the dnn based on QpNet for DQN.
    
    The struture is FC-ReLU-BN-FC-ReLU-BN-multi_QP.
    """
    
    def __init__(self, num_input, num_output,num_hidden, num_z=5, num_ineq=3*2*5,
                 num_equ= 4*5,eps = 1e-4):
        """Inits the class.
        num_z: (# action variable) * horizon length
        num_ineq >= (# ineq constaints) * horizon length
        num_equ >= (# eq constaints) * horizon length
        """
        super().__init__()
        self.eps = eps
        self.normal_layers = nn.Sequential(
            nn.Linear(num_input, num_hidden),
            nn.ReLU(),
            nn.BatchNorm1d(num_hidden),
            nn.Linear(num_hidden, num_z),
            nn.ReLU(),
            nn.BatchNorm1d(num_z)
        )
        
        # QP params for multiple QP layers
        self.M = Variable(torch.tril(torch.ones(num_z, num_z)))
        self.I = Variable(torch.eye(num_z))
        self.L = []
        self.G = []
        self.A = []
        self.z0 = []
        self.s0 = []
        for i in range(2):
            self.L.append(Parameter(torch.tril(torch.rand(num_z, num_z))))
            self.G.append(Parameter(torch.Tensor(num_ineq,num_z).uniform_(-1,1)))
            self.A.append(Parameter(torch.Tensor(num_equ,num_z).uniform_(-1,1)))
            self.z0.append(Parameter(torch.zeros(num_z)))
            self.s0.append(Parameter(torch.ones(num_ineq)))
                
    def forward(self, x):
        """Builds the forward strucre of the QPNet.
        """
        #print('f,input',x.size())
        x = self.normal_layers(x)
        #print('f,nz',x.size())
        # Set up the qp parameters Q=L*L^T+ϵ*I, h = G*z_0+s_0, b = A*z0.
        Qp_list = []
        for i in range(2):
            L = self.M*self.L[i]
            Q = L.mm(L.t()) + self.eps*self.I
            h = self.G[i].mv(self.z0[i])+self.s0[i]
            b = self.A[i].mv(self.z0[i])
            z_opt = QPFunction(verbose=-1)(Q, x, self.G[i], h, self.A[i], b)
            Qp_value = (z_opt.mm(Q)*z_opt/2 + x*z_opt).sum(1).unsqueeze(1)
            #print('forward: Qp_value',Qp_value.size())
            # 1/2*z^T*Q*z + p^T*z
            Qp_list.append(Qp_value)
        x = torch.cat(Qp_list,1)
        #print('forward:Qp_2action',x.size())
        
        return x
    
    def act(self, state, epsilon):
        """The action excuted by epsilon-greedy exploration
        """
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(state).unsqueeze(0)) # adds extra dim at front
            q_value = self.forward(state)
            #print('act:q_value ',q_value)
            action  = q_value.max(1)[1]
            #print('act:model action ',action)
        else:
            action = random.randrange(env.action_space.n)
        return action

In [12]:
random.randrange(env.action_space.n)

1

In [13]:
env = MyEnv()
model = DQN(4, env.action_space.n,20)

model = model.cuda()
model.eval()    
optimizer = optim.Adam(model.parameters())

In [26]:
state, action, reward, next_state, done = replay_buffer.sample(batch_size)
state1      = Variable(torch.FloatTensor(state))

In [36]:
np.shape(state)
state1.size()

torch.Size([32, 4])

In [14]:
def compute_td_loss(replay_buffer,batch_size):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)
    print(state.size)
    state      = Variable(torch.FloatTensor(state))
    print(state.size)
    next_state = Variable(torch.FloatTensor(next_state))
    action     = Variable(torch.LongTensor(action))
    reward     = Variable(torch.FloatTensor(reward))
    done       = Variable(torch.FloatTensor(done))

    q_values      = model(state)
    next_q_values = model(next_state)
    #print('loss:q_values',q_values.size())
    #print('loss:action', action.size())
    q_value          = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
    next_q_value     = next_q_values.max(1)[0]
    expected_q_value = reward + gamma * next_q_value * (1 - done)
    loss = (q_value - Variable(expected_q_value.data)).pow(2).mean()
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss

In [18]:
num_run_period = []  # records the # periods in one trajectory
t = 0
num_frames = 33
batch_size = 32
replay_buffer = ReplayBuffer(2 * batch_size)  #2 possible actions each round, batch_size rounds per batch
gamma      = 0.99
losses = []
episode_reward = 0
avg_loss = 0
avg_losses = []
state = env.reset()
for frame_idx in range(1, num_frames + 1):
    t = t+1
    env.render()
    epsilon = epsilon_by_frame(frame_idx)
    action = model.act(state, epsilon)
    next_state, reward, done, _ = env.step(int(action))
    replay_buffer.push(state, action, reward, next_state, done)
    
    state = next_state
    episode_reward += reward
    #env.render()
    if done:
        state = env.reset()
        num_run_period.append(t)
        t = 0
    
    if len(replay_buffer) > batch_size:
        loss = compute_td_loss(replay_buffer,batch_size)
        losses.append(loss.data.item())
        avg_loss = avg_loss + loss.data.item()
        avg_losses.append(avg_loss/frame_idx)
#     try:
#         avg_loss = avg_loss + loss.data.item()
#     except:
#         pass
    
    if frame_idx % 50 == 0:
        clear_output(True)
        plt.figure(figsize=(20,5))
        plt.subplot(131)
        plt.plot(losses)
        plt.subplot(132)
        plt.plot(avg_losses)
        plt.show()
env.close()

128
<built-in method size of Tensor object at 0x7f4117fd5798>


In [29]:
loss.data

tensor(1.0000, device='cuda:0')

In [12]:
num_run_period

[9,
 10,
 8,
 10,
 10,
 8,
 9,
 9,
 10,
 9,
 11,
 9,
 10,
 9,
 9,
 9,
 10,
 10,
 9,
 10,
 9,
 9,
 10,
 10,
 8,
 10,
 10,
 10,
 9,
 10,
 8,
 8,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 10,
 10,
 9,
 9,
 10,
 10,
 10,
 10,
 9,
 10,
 10,
 10,
 9,
 9,
 8]

In [60]:
a = torch.tensor([2,3]).unsqueeze(1)
b = torch.tensor([1,1]).unsqueeze(1)
c = torch.cat([a,b],1)
c

tensor([[2, 1],
        [3, 1]])

In [13]:
env.close()